#Paquetes necesarios

In [7]:
import cv2  
import matplotlib.pyplot as plt
from ultralytics import YOLO
import pytesseract
from pytesseract import Output 
import re

Desde cámara, detección con yolov8 y modelo nano. Visualización propia

In [55]:
# Tamaño deseado para mostrar las imágenes
desired_size = (800, 500)

# Ruta del modelo YOLO
model = YOLO('matricula.pt')  # Asegúrate de tener la implementación de YOLO cargada

# Ruta absoluta de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract'

def show_image(window_name, image):
    resized_image = cv2.resize(image, desired_size)
    cv2.imshow(window_name, resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Leer la imagen
image_path = "coche5.jpeg"
car = cv2.imread(image_path)

# Detectar matrículas
results = model(car, verbose=False)

# Para cada detección
for r in results:
    boxes = r.boxes

    for box in boxes:
        # Coordenadas del contenedor
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # convertir a valores enteros
        
        # Tenemos solo con la matrícula
        plate = car[y1:y2, x1:x2]
        
        # Recuadro de la matrícula
        cv2.rectangle(car, (x1, y1), (x2, y2), (0, 0, 255), 3)
        
        # Definir el patrón de la matrícula "0000 AAA"
        plate_pattern = re.compile(r'^\d{4}\s[A-Z]{3}$')
        
        # Aplicamos OCR a la matrícula
        text = pytesseract.image_to_string(plate, config='--psm 8', output_type=Output.STRING)
        
        if plate_pattern.match(text):
        
            print("Matricula detectada:", text)
        else:
            text = re.sub(r'[^A-Za-z0-9]', ' ', text)
            print("Matricula detectada:", text)

        # Pasamos a RGB el coche y de la matrícula 
        car_rgb = cv2.cvtColor(car, cv2.COLOR_BGR2RGB)
        plate_rgb = cv2.cvtColor(plate, cv2.COLOR_BGR2RGB)
 
        show_image("Coche con la matrícula marcada", car_rgb)
        show_image("Matrícula", plate_rgb)
        break

Matricula detectada:   BG2I KYX   
